# Imports

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import torch.nn.functional as F

<IPython.core.display.Javascript object>

In [3]:
import torch
torch.__version__

'1.9.0'

<IPython.core.display.Javascript object>

# Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [7]:
#export
import sys
sys.path.append('../')
from exp.nb_02 import *


def get_data():
    """Download MNIST dataset and return training and validation tensors."""
    path = Path("/Users/imad/.fastai/data/mnist.pkl.gz")

    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

    return map(tensor, (x_train, y_train, x_valid, y_valid))


def normalize(x, mean, std):
    """Normalize x by first centering it and then scale it."""
    return (x - mean) / std

<IPython.core.display.Javascript object>

In [8]:
torch.nn.modules.conv._ConvNd.reset_parameters??

<IPython.core.display.Javascript object>

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self) -> None
Docstring: <no docstring>
Source:   
    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/envs/fastbook/lib/python3.9/site-packages/torch/nn/modules/conv.py
Type:      function


In [9]:
x_train, y_train, x_valid, y_valid = get_data()
train_mean, train_std = x_train.mean(), x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

<IPython.core.display.Javascript object>

In [10]:
# Reshape to C x h x w to use Conv layers because Pytorch use b x C x H x W
x_train = x_train.view(-1, 1, 28, 28)
x_valid = x_valid.view(-1, 1, 28, 28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

<IPython.core.display.Javascript object>

In [11]:
n, *_ = x_train.shape
c = y_train.max() + 1
nh = 32
n, c

(50000, tensor(10))

<IPython.core.display.Javascript object>

In [12]:
l1 = nn.Conv2d(1, nh, 5)

<IPython.core.display.Javascript object>

In [13]:
x = x_valid[:100]

<IPython.core.display.Javascript object>

In [14]:
x.shape

torch.Size([100, 1, 28, 28])

<IPython.core.display.Javascript object>

In [15]:
def stats(x): 
    return x.mean(), x.std()

<IPython.core.display.Javascript object>

- weight shape in Conv2D is `out_channels x input_channels x filter_height x filter_width`.

In [16]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

<IPython.core.display.Javascript object>

In [17]:
stats(l1.weight), stats(l1.bias)

((tensor(0.0015, grad_fn=<MeanBackward0>),
  tensor(0.1129, grad_fn=<StdBackward>)),
 (tensor(0.0179, grad_fn=<MeanBackward0>),
  tensor(0.1361, grad_fn=<StdBackward>)))

<IPython.core.display.Javascript object>

In [18]:
t = l1(x)
stats(t)

(tensor(0.0212, grad_fn=<MeanBackward0>),
 tensor(0.7140, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

- Kaiming normal is generally used when the activation function is either **Relu** or **Leaky Reu**.
- Since all pixels are nonnegative, so leaky relu is the same as **a = 1**.

In [19]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(-0.0017, grad_fn=<MeanBackward0>),
 tensor(1.1288, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

Looks like it is working.

In [20]:
def f1(x, a=0):
    return F.leaky_relu(l1(x), a)

<IPython.core.display.Javascript object>

In [21]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5847, grad_fn=<MeanBackward0>),
 tensor(1.0844, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

The mean is no longer zero because after normalizing the values and using Relu, we removed all values below zero and assign them a zero value. Therefore, the mean will no longer be zero.

In [22]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2177, grad_fn=<MeanBackward0>),
 tensor(0.3758, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

The standard deviation of default initialization is about **0.36** which is not good at all.

In [23]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

<IPython.core.display.Javascript object>

In [24]:
# receptive field size (filter_height x filter_width)
rec_fs = l1.weight[0, 0].numel()
rec_fs

25

<IPython.core.display.Javascript object>

In [25]:
nf, ni= l1.weight.shape[:2]
nf,ni

(32, 1)

<IPython.core.display.Javascript object>

In [26]:
fan_in  = ni * rec_fs
fan_out = nf * rec_fs
fan_in, fan_out

(25, 800)

<IPython.core.display.Javascript object>

$$gain = \sqrt\frac{2}{1 + a ^2}$$

In [27]:
def gain(a):
    return math.sqrt(2.0 / (1 + a ** 2))

<IPython.core.display.Javascript object>

$a = 0$ is Relu.

In [28]:
gain(0), gain(0.01), gain(0.1), gain(1), gain(math.sqrt(5.))

(1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 1.0,
 0.5773502691896257)

<IPython.core.display.Javascript object>

The first 3 gains are close to $\sqrt{2}$.

PyTorch uses *kaiming_uniform* for initializing the weight tensors of Conv2D layer. The mean and std of a uniform distribution are $\frac{(b + a)}{2}$ and $\sqrt{\frac{(b - a)^2}{12}}$.

In [48]:
(1 - (-1)) / math.sqrt(12)

0.5773502691896258

<IPython.core.display.Javascript object>

The gain of $\sqrt{5}$ is the same as the std of uniform[-1, 1].

In [49]:
torch.zeros(10000).uniform_(-1, 1).std()

tensor(0.5790)

<IPython.core.display.Javascript object>

In [50]:
def kaiming2(x, a, use_fan_out=False):
    nf, ni, *_ = x.shape
    rec_fs = x[0, 0].shape.numel()
    fan = nf * rec_fs if use_fan_out else ni * rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound, bound)

<IPython.core.display.Javascript object>

since $std = \sqrt{\frac{(b - a)^2}{12}}$ and assume $a = -b$; therefore, $std = \sqrt\frac{4b^2}{12} = \frac{b}{\sqrt{3}}$
$$\rightarrow bound = std * \sqrt{3}$$

In [53]:
stats(l1.weight)

(tensor(-0.0176, grad_fn=<MeanBackward0>),
 tensor(0.2806, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

In [54]:
kaiming2(l1.weight, a=0);
stats(l1.weight)

(tensor(8.2925e-05, grad_fn=<MeanBackward0>),
 tensor(0.2770, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

In [32]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5250, grad_fn=<MeanBackward0>),
 tensor(0.9223, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

In [33]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2238, grad_fn=<MeanBackward0>),
 tensor(0.4244, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

In [34]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(-1)

<IPython.core.display.Javascript object>

In [35]:
m = nn.Sequential(
    nn.Conv2d(1, 8, 5, stride=2, padding=2), nn.ReLU(),
    nn.Conv2d(8, 16, 3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 32, 3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(32, 1, 3, stride=2, padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

<IPython.core.display.Javascript object>

In [36]:
y = y_valid[:100].float()

<IPython.core.display.Javascript object>

In [37]:
t = m(x)
stats(t)

(tensor(-0.0178, grad_fn=<MeanBackward0>),
 tensor(0.0115, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

That is bad because the input had a std = 1, the first hidden layer had std = 0.4 and the 4th layer had a std ~ 0. That is not good at all because that means you can't learn anything with 4 layers and you can't create a model with more layers due to this issue.

In [38]:
l = mse(t,y)
l.backward()

<IPython.core.display.Javascript object>

In [39]:
stats(m[0].weight.grad)

(tensor(-0.0008), tensor(0.0379))

<IPython.core.display.Javascript object>

The gradients also have close to zero std.

In [41]:
init.kaiming_uniform_??

<IPython.core.display.Javascript object>

Signature:
init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights

In [42]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

<IPython.core.display.Javascript object>

In [43]:
t = m(x)
stats(t)

(tensor(0.1387, grad_fn=<MeanBackward0>),
 tensor(0.2490, grad_fn=<StdBackward>))

<IPython.core.display.Javascript object>

In [44]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(-0.1562), tensor(0.5715))

<IPython.core.display.Javascript object>

In [55]:
nn.init.calculate_gain??

<IPython.core.display.Javascript object>

Signature: nn.init.calculate_gain(nonlinearity, param=None)
Source:   
def calculate_gain(nonlinearity, param=None):
    r"""Return the recommended gain value for the given nonlinearity function.
    The values are as follows:

    ================= ====================================================
    nonlinearity      gain
    ================= ====================================================
    Linear / Identity :math:`1`
    Conv{1,2,3}D      :math:`1`
    Sigmoid           :math:`1`
    Tanh              :math:`\frac{5}{3}`
    ReLU              :math:`\sqrt{2}`
    Leaky Relu        :math:`\sqrt{\frac{2}{1 + \text{negative\_slope}^2}}`
    SELU              :math:`\frac{3}{4}`
    ================= ====================================================

    .. warning::
        In order to implement `Self-Normalizing Neural Networks`_ ,
        you should use ``nonlinearity='linear'`` instead of ``nonlinearity='selu'``.
        This gives the initial weights a variance of `

The above gradients and weights std are much better in terms of std and are no longer close to zero.

# Export

In [45]:
!python ../src/notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py


<IPython.core.display.Javascript object>